# Laboratorio 2 - Agrupación

### Integrantes

1. Cesar Avellaneda, 202214746, c.avellanedac@uniandes.edu.co.
2. Santiago Tinjaca, 202215991, s.tinjaca@uniandes.edu.co.
3. Jorge Bustamante, 202210581, j.bustamantep@uniandes.edu.co.

Este notebook tiene los siguientes elementos: 
1. Cargue de los datos.

2. Entendimiento de los datos: Describir las características más relevantes de los datos y todo el perfilamiento de datos, incluir el análisis de calidad de datos y hacer una preselección de las variables más importantes para la etapa de modelado.

3. Preparación de datos: Solucionar los problemas de calidad de datos previamente identificados que afecten el modelo a construir. Además, debe aplicar todos los proceso de preprocesamiento de datos necesarios para la construcción del modelo de agrupación.

4. Modelado: Utilizando las variables previamente seleccionadas, construir un modelo de agrupación que tenga siluetas cercanas a 1.

5. Evaluación cuantitativa: A partir de las métricas seleccionadas para evaluar y seleccionar el mejor modelo, explicar el resultado obtenido desde el punto de vista cuantitativo. Contestar a la pregunta: ¿Su equipo recomienda utilizar en producción el modelo de agrupación para optimizar tiempos? ¿Por qué? En caso de no recomendar el uso del modelo, ¿qué recomendaciones haría para continuar iterando con el objetivo de la construcción de un mejor modelo?

6. Evaluación cualitativa: Interpretación de los grupos obtenidos y relación entre estos y el objetivo de la organización. Verificar si es posible usar los datos obtenidos para optimizar la toma de decisiones.

### Entendimiento del negocio:
El caso de estudio es de un hospital que haciendo uso de la metodología KTAS quiere solicitar un modelo que pueda agrupar pacientes por ciertas características en común y le ayude al hospital a optimizar sus tiempos de respuesta.
### Enfoque Analítico:
En este laboratorio vamos a hacer un modelo predictivo usando un aprendizaje supervisado y un modelo de agrupación para hacer uso de las condiciones de llegada de los pacientes y optimizar las decisiones tomadas.

### 1. Carga de datos

In [440]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

In [441]:
datos=pd.read_csv('./data/202420_Laboratorio 2 - Agrupación_202420_Laboratorio_2_-_Agrupación_data.csv', sep=',', encoding = 'utf-8')

### 2. Entendimiento de los datos

#### 2.1 Perfilamiento de los datos

In [ ]:
datos.shape

In [ ]:
datos.dtypes

In [ ]:
datos.sample(5)

In [ ]:
datos.info()

In [ ]:
datos.describe()

In [ ]:
#Visualización de todas las variables numéricas
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos, orient="v")

In [ ]:
datos["Lesion_stan"]=datos["Lesion"].apply(lambda x:0 if x == 2 else x)
datos["Lesion_stan"].value_counts()
name_cols_float = datos.select_dtypes(include = ['float']).columns
name_cols_int = datos.select_dtypes(include = ['int64']).columns
name_cols_int

In [ ]:
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos[name_cols_float], orient="v")

In [ ]:
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos[name_cols_int], orient="v")

In [ ]:
name_cols_non_number = datos.select_dtypes(include = ['object']).columns
name_cols_non_number

In [ ]:
datos.Queja_Principal.sample(10)

In [ ]:
datos[name_cols_non_number].describe()

In [ ]:
datos[name_cols_non_number].sample(15)

### 2.2 Análisis de calidad de datos

#### 2.2.1 Análisis de completitud

In [ ]:
datos.isnull().sum() / datos.shape[0]

#### 2.2.2 Unicidad

No podemos eliminar los datos sin más porque nada nos asegura que sean duplicados, puede ser que hayan dos pacientes que tengan los mismos problemas al momento de llegar al hospital.

In [ ]:
datos.duplicated(keep = False).sum()

#### 2.2.3 Consistencia

In [ ]:
datos.Estado_Mental.value_counts()

In [ ]:
#Visualización de la variable dolor registrado por la enfermera
datos.dolor_NRS.value_counts()

In [ ]:
datos.KTAS_experto.value_counts()

Como podemos observar, dolor_NRS tiene un problema. Hay datos con el valor #BOÞ!, los cuales representan una gran cantida del total.

#### 2.2.4 Validez

In [ ]:
datos['KTAS_experto'].unique()

In [ ]:
datos['dolor_NRS'].unique()

In [ ]:
datos['Estado_Mental'].unique()

Como podemos observar, la validez de estos datos es correcta, el único problema como mencionamos anteriormente está en dolor_NRS

### 3. Preparación de datos

Dato que los datos atipicos de duración de instancia en minutos están bastante arriba a diferencia de los demás registros, decidimos escoger solo los datos que están por debajo de 850. 

In [463]:
datos=datos[datos["Duracion_Estancia_Min"]<=850]

In [ ]:
def calcularEWS(registro):
    total = 0
    
    # Frecuencia respiratoria (RR)
    if registro['RR'] <= 8:
        total += 2
    elif 9 <= registro['RR'] <= 14:
        total += 0
    elif 15 <= registro['RR'] <= 20:
        total += 1
    elif 21 <= registro['RR'] <= 29:
        total += 2
    elif registro['RR'] >= 30:
        total += 3
    
    # Presión arterial sistólica (SBP)
    if registro['SBP'] <= 70:
        total += 3
    elif 71 <= registro['SBP'] <= 80:
        total += 2
    elif 81 <= registro['SBP'] <= 100:
        total += 1
    elif 101 <= registro['SBP'] <= 199:
        total += 0
    elif registro['SBP'] >= 200:
        total += 2

    # Frecuencia cardíaca (HR)
    if registro['HR'] <= 40:
        total += 2
    elif 41 <= registro['HR'] <= 50:
        total += 1
    elif 51 <= registro['HR'] <= 100:
        total += 0
    elif 101 <= registro['HR'] <= 110:
        total += 1
    elif 111 <= registro['HR'] <= 129:
        total += 2
    elif registro['HR'] >= 130:
        total += 3

    # Temperatura corporal (BT)
    if registro['BT'] < 35.0:
        total += 2
    elif 35.0 <= registro['BT'] <= 38.4:
        total += 0
    elif 38.5 <= registro['BT']:
        total += 2

    # Saturación de oxígeno (Saturacion)
    if registro['Saturacion'] <= 91:
        total += 3
    elif 92 <= registro['Saturacion'] <= 93:
        total += 2
    elif 94 <= registro['Saturacion'] <= 95:
        total += 1

    # Nivel de conciencia
    if registro['Estado_Mental'] == 1:
        total += 0
    elif registro['Estado_Mental'] == 2:
        total += 1
    elif registro['Estado_Mental'] == 3:
        total += 2
    elif registro['Estado_Mental'] == 4:
        total += 3
    return total

datos['EWS'] = datos.apply(calcularEWS, axis=1)
print(datos.head())

In [ ]:
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos[name_cols_int], orient="v")

Se escogen las variables KTAS_experto, Estado_Mental y dolor_NRS porque se cree que podrían agrupar a los pacientes de una buena forma. Dado que la valoración dada por el experto puede relacionarse con el nivel de dolor y el estado mental en el que llega el paciente de manera que, si el dolor registrado por la enfermera es alto y el estado mental del paciente es malo, la valoración del experto será más o menos grave.

In [ ]:

gravedad_dict = {
    1: 1,  # Alta a Domicilio
    2: 2,  # Admisión a Sala
    7: 3,  # Cirugía
    4: 4,  # Alta a otra Institución de Cuidados
    5: 5,  # Transferencia a otro Hospital
    3: 6,  # Admisión a UCI
    6: 7   # Muerte
}

# Crear una nueva columna con la gravedad ajustada
datos['gravedad'] = datos['Disposicion'].map(gravedad_dict)

datos['dolor_NRS'] = pd.to_numeric(datos['dolor_NRS'], errors='coerce')
datos['dolor_NRS'] = datos['dolor_NRS'].fillna(0)
datos["Duracion_KTAS_Min"] = datos["Duracion_KTAS_Min"].str.replace(',', '.').astype('float64')


In [ ]:

name_cols = ['KTAS_experto', 'EWS' ,'dolor_NRS']
datos_cols_selec = datos[name_cols].copy()
datos_cols_selec.describe()


Problemas de calidad:

Primero el dolor reportado por la enfermera, el cual tiene un 44% de sus entradas en null.
Nos dimos cuenta de que estas se correspondían cuando el paciente no tenía dolor, por lo que las asignamos a 0.

In [ ]:

datos_cols_selec = datos_cols_selec.reset_index(drop=True)
datos_cols_selec.describe()


In [ ]:
fig=plt.figure(figsize=(20,10))
ax = sns.boxplot(data=datos_cols_selec, orient="v")

In [600]:
# sns.pairplot(data=datos_cols_selec, hue="dolor_NRS")

### 4. Modelamiento

In [ ]:
mms = MinMaxScaler()


datos_prep_norm = datos_cols_selec.copy()
datos_prep_norm[name_cols] = mms.fit_transform(datos_cols_selec[name_cols])
datos_prep_norm=datos_prep_norm[name_cols]
saved_cols = datos_prep_norm.columns

datos_prep_norm = pd.DataFrame(datos_prep_norm, columns =saved_cols)
print(datos_prep_norm.head())

In [ ]:
datos_prep_norm.describe()

In [ ]:
fig=plt.figure(figsize=(20,8))
ax = sns.boxplot(data=datos_prep_norm, orient="v") 

#### 4.1 Encontramos el número óptimo de clusters con el método del codo

In [604]:
def plot_distortion(data,
                    k_min=1, 
                    k_max=11,
                    ylabel = 'Distortion',
                    xlabel = 'Number of clusters',
                    title = 'Distortion Plot'):
    '''
    Graficar el codo de los clusters
    
    Parametros
    ----------
    data : np.array
        El arreglo con los datos
    k_min : int
        Valor mínimo para k
    k_max : int
        Valor máximo para k
    xlabel : string
        La etiqueta del eje x
    ylabel  string
        La etiqueta del eje y    
    title : string
        El titulo de la gráfica  
    '''
    distortions = []
    for i in range(k_min, k_max):
        km = KMeans(n_clusters=i,
                 init='k-means++',
                 n_init=10,
                 max_iter=300,
                 random_state=0)
        km.fit(data)
        distortions.append(km.inertia_)
    plt.plot(range(k_min,k_max), distortions, marker='o')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
plot_distortion(datos_prep_norm,1,11)

#### 4.2 Construir el nuevo modelo con el número de clusters obtenido

In [606]:
N_clusters=2
kmeans = KMeans(n_clusters=N_clusters, random_state=0) 
kmeans = kmeans.fit(datos_prep_norm)

#### 4.3 Visualizar el resultado

#### 4.3.1 Graficar cantidad de registros por agrupación

In [ ]:
labels = kmeans.labels_
datos_prep_norm['Cluster'] = labels

cluster_distrib = datos_prep_norm['Cluster'].value_counts()

fig=plt.figure(figsize=(12,8))
sns.barplot(x=cluster_distrib.index, y=cluster_distrib.values, color='b')

In [ ]:
cols_number = datos_prep_norm.to_numpy()
datos_prep_norm.groupby('Cluster').count()

#### 4.3.2  Graficar comportamiento en parejas de atributos de acuerdo a la agrupación

In [ ]:
sns.pairplot(data=datos_prep_norm, hue="Cluster", palette="Dark2")

####  4.3.3  Graficar dos atributos y ver la relación con las agrupaciones

In [ ]:
plt.scatter(cols_number[kmeans.labels_ == 0, 0], cols_number[kmeans.labels_ == 0, 1], s = 100, marker='v', c = 'red', label = 'Cluster 1')
plt.scatter(cols_number[kmeans.labels_ == 1, 0], cols_number[kmeans.labels_ == 1, 1], s = 100, marker='o', c = 'blue', label = 'Cluster 2')
plt.scatter(cols_number[kmeans.labels_ == 2, 0], cols_number[kmeans.labels_ == 2, 1], s = 100, marker='*', c = 'green', label = 'Cluster 3')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 100, c = 'yellow', label = 'Centroids')
plt.title('Clusters')
plt.xlabel(name_cols[0])
plt.ylabel(name_cols[1])
plt.legend()
plt.show()

#### 4.3.4 Evaluar la calidad de los clústeres obtenidos

In [611]:
def plot_silhouette(data, 
                    labels,
                   ylabel = 'Grupos',
                   xlabel = "Coeficiente de silueta",
                   title = 'Gráfica de silueta'):
    '''
    Graficar la silueta de los clusters
    
    Parametros
    ----------
    data : np.array
        El arreglo con los datos
    labels : np.array
        El arreglo con las etiquetas correspondientes
    ylabel  string
        La etiqueta del eje y
    xlabel : string
        La etiqueta del eje x
    title : string
        El titulo de la gráfica        
    '''
    cluster_labels = np.unique(labels)
    print(cluster_labels)
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = silhouette_samples(data,
                                        labels,
                                        metric='euclidean')
    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    for i, c in enumerate(cluster_labels):
        c_silhouette_vals = silhouette_vals[labels == c]
        c_silhouette_vals.sort()
        y_ax_upper += len(c_silhouette_vals)
        color = cm.jet(float(i) / n_clusters)
        plt.barh(range(y_ax_lower, y_ax_upper),
                        c_silhouette_vals,
                        height=1.0,
                        edgecolor='none',
                        color=color)
        yticks.append((y_ax_lower + y_ax_upper) / 2.)
        y_ax_lower += len(c_silhouette_vals)
    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg,
                color="red",
                linestyle="--")
    plt.yticks(yticks, cluster_labels+1)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.title(title)
    plt.show()

In [ ]:
plot_silhouette(data = datos_prep_norm, 
                labels = kmeans.labels_, 
                ylabel = 'Modelo de dos Agrupaciones')

## 5. Análisis y conclusiones

In [613]:
name_cols = ['KTAS_experto', 'EWS' ,'dolor_NRS']
datos_cols_selec = datos[name_cols].copy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

def plot_dbscan_eps(data, eps_min=0.1, eps_max=2.0, step=0.1, min_samples=5):
    '''
    Graficar el número de clústeres y ruido vs. eps en DBSCAN
    
    Parámetros
    ----------
    data : np.array
        El arreglo con los datos
    eps_min : float
        Valor mínimo para eps
    eps_max : float
        Valor máximo para eps
    step : float
        El incremento de eps en cada iteración
    min_samples : int
        Número mínimo de muestras para ser un punto central
    '''
    eps_values = np.arange(eps_min, eps_max, step)
    n_clusters = []
    n_noises = []

    for eps in eps_values:
        db = DBSCAN(eps=eps, min_samples=min_samples)
        db.fit(data)
        
        labels = db.labels_
        # Número de clústeres (excluyendo el ruido, que es etiquetado como -1)
        n_clusters.append(len(set(labels)) - (1 if -1 in labels else 0))
        # Contar cuántos puntos fueron etiquetados como ruido (-1)
        n_noises.append(list(labels).count(-1))
    
    # Graficar el número de clústeres y puntos de ruido
    fig, ax1 = plt.subplots()

    color = 'tab:blue'
    ax1.set_xlabel('Eps')
    ax1.set_ylabel('Número de Clústeres', color=color)
    ax1.plot(eps_values, n_clusters, marker='o', color=color, label='Número de Clústeres')
    ax1.tick_params(axis='y', labelcolor=color)
    
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Número de Puntos de Ruido', color=color)
    ax2.plot(eps_values, n_noises, marker='x', color=color, label='Número de Ruido')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()
    plt.title('Número de Clústeres y Ruido vs. Eps')
    plt.show()

scaler = StandardScaler()
df_scaled = scaler.fit_transform(datos_cols_selec)
plot_dbscan_eps(df_scaled, eps_min=0.1, eps_max=2, step=0.1, min_samples=10)


In [615]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Escalar los datos para normalizar distancias (recomendado para DBSCAN)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(datos_cols_selec)


In [ ]:

# Crear el modelo DBSCAN con los parámetros: eps y min_samples
dbscan = DBSCAN(eps=1, min_samples=10)

# Ajustar el modelo a los datos escalados
dbscan.fit(datos_cols_selec)

print(pd.DataFrame(dbscan.labels_).value_counts())

In [ ]:

# Agregar las etiquetas al DataFrame
datos_cols_selec['cluster'] = dbscan.labels_

# Mostrar los resultados
print(datos_cols_selec['cluster'].value_counts())


In [ ]:
sns.pairplot(data=datos_cols_selec, hue="cluster", palette="Dark2")

In [ ]:
plot_silhouette(data = datos_cols_selec, 
                labels = dbscan.labels_, 
                ylabel = 'Modelo de dos Agrupaciones')

In [691]:
name_cols = [ "Grupo", 
              "Disposicion",
              "KTAS_experto",
              "EWS",
              "dolor_NRS"]
name_cols = ['KTAS_experto', 'dolor_NRS', 'EWS']
datos_cols_selec = datos[name_cols].copy()


In [ ]:
mms = MinMaxScaler()


datos_prep_norm = datos_cols_selec.copy()
datos_prep_norm[name_cols] = mms.fit_transform(datos_cols_selec[name_cols])
datos_prep_norm=datos_prep_norm[name_cols]
saved_cols = datos_prep_norm.columns

datos_prep_norm = pd.DataFrame(datos_prep_norm, columns =saved_cols)
print(datos_prep_norm.head())

In [693]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering

def plot_agglomerative_clusters(data, k_min=2, k_max=10, linkage_method='ward'):
    '''
    Graficar el número de clústeres usando Agglomerative Clustering
    
    Parámetros
    ----------
    data : np.array
        El arreglo con los datos
    k_min : int
        Valor mínimo para n_clusters
    k_max : int
        Valor máximo para n_clusters
    linkage_method : string
        El método de linkage que debe usar (ward, single, complete, average)
    '''
    cluster_counts = []
    
    for n_clusters in range(k_min, k_max + 1):
        # Crear el modelo de Agglomerative Clustering
        agg_clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage_method)
        
        # Ajustar el modelo a los datos
        agg_clustering.fit(data)
        # Almacenar el número de clústeres formados
        cluster_counts.append(silhouette_score(data, agg_clustering.labels_))
    
    # Graficar el número de clústeres
    plt.plot(range(k_min, k_max + 1), cluster_counts, marker='o')
    plt.xlabel('Número de Clústeres (k)')
    plt.ylabel('Silueta')
    plt.title(f'Agglomerative Clustering con {linkage_method.capitalize()} Linkage')
    plt.show()


In [ ]:

# Usar la función para graficar el número de clústeres formados
plot_agglomerative_clusters(datos_prep_norm, k_min=2, k_max=6, linkage_method='ward')


In [ ]:
plot_agglomerative_clusters(datos_prep_norm, k_min=2, k_max=6, linkage_method='complete')


In [ ]:
plot_agglomerative_clusters(datos_prep_norm, k_min=2, k_max=6, linkage_method='average')


In [ ]:
plot_agglomerative_clusters(datos_prep_norm, k_min=2, k_max=6, linkage_method='single')


In [ ]:
agg_clustering = AgglomerativeClustering(n_clusters=3, linkage='ward')
        
# Ajustar el modelo a los datos
agg_clustering.fit(datos_prep_norm)



In [ ]:
# Agregar las etiquetas al DataFrame
datos_prep_norm['cluster'] = agg_clustering.labels_

# Mostrar los resultados
print(datos_prep_norm['cluster'].value_counts())

In [ ]:
sns.pairplot(data=datos_prep_norm, hue="cluster", palette="Dark2")

In [ ]:
silhouette_avg = silhouette_score(datos_prep_norm, labels)
print(f"Silhouette Score: {silhouette_avg}")

In [ ]:
plot_silhouette(data = datos_prep_norm, 
                labels = agg_clustering.labels_, 
                ylabel = 'Modelo de dos Agrupaciones')